In [1]:
import pandas as pd
import gzip
import math
import random
from collections import defaultdict
import ast

In [2]:
#input files: PP_recipes.csv, RAW_interactions.csv  ingr_map.pkl           interactions_train.csv
#PP_users.csv    RAW_recipes.csv       interactions_test.csv  interactions_validation.csv

In [3]:
df_recipes = pd.read_csv('/home/prpathak/.cache/kagglehub/datasets/shuyangli94/food-com-recipes-and-user-interactions/versions/2/PP_recipes.csv')

In [4]:
df_recipes.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [5]:
df_raw_recipes = pd.read_csv('/home/prpathak/.cache/kagglehub/datasets/shuyangli94/food-com-recipes-and-user-interactions/versions/2/RAW_recipes.csv')


In [6]:
df_raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [7]:
df_raw_interactions = pd.read_csv('/home/prpathak/.cache/kagglehub/datasets/shuyangli94/food-com-recipes-and-user-interactions/versions/2/RAW_interactions.csv')


In [8]:
df_raw_interactions.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [9]:
df_users = pd.read_csv('/home/prpathak/.cache/kagglehub/datasets/shuyangli94/food-com-recipes-and-user-interactions/versions/2/PP_users.csv')


In [10]:
df_users.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [11]:
df_interactions_train = pd.read_csv('/home/prpathak/.cache/kagglehub/datasets/shuyangli94/food-com-recipes-and-user-interactions/versions/2/interactions_train.csv')

In [12]:
df_interactions_train.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [13]:
direc = '/home/prpathak/.cache/kagglehub/datasets/shuyangli94/food-com-recipes-and-user-interactions/versions/2/'

df_interactions_train = pd.read_csv(direc+"interactions_train.csv")
df_interactions_valid = pd.read_csv(direc+"interactions_validation.csv")
df_interactions_test = pd.read_csv(direc+"interactions_test.csv")

In [118]:
import scipy

userIDs,itemIDs = {},{}

for ind, d in df_interactions_train.iterrows():
    u,i = d['user_id'], d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
for ind, d in df_interactions_valid.iterrows():
    u,i = d['user_id'], d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
for ind, d in df_interactions_test.iterrows():
    u,i = d['user_id'], d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
nUsers = len(userIDs)
nItems = len(itemIDs)

In [119]:
X = scipy.sparse.lil_matrix((len(df_interactions_train)+len(df_interactions_valid)+len(df_interactions_test), nUsers+nItems))

In [120]:
X

<718379x203341 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [121]:
for i in range (len(df_interactions_train)):
    user = userIDs[df_interactions_train.loc[i]['user_id']]
    item = itemIDs[df_interactions_train.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
for i in range (len(df_interactions_valid)):
    user = userIDs[df_interactions_valid.loc[i]['user_id']]
    item = itemIDs[df_interactions_valid.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
for i in range (len(df_interactions_test)):
    user = userIDs[df_interactions_test.loc[i]['user_id']]
    item = itemIDs[df_interactions_test.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1

In [122]:
import numpy as np
y = np.array([d['rating'] for _,d in df_interactions_train.iterrows()])

In [123]:
from fastFM import als
from scipy.spatial import distance
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)

In [124]:
train_ind = int(X.shape[0]*0.5)
train_ind

359189

In [125]:
X_train,y_train = X[:train_ind],y[:train_ind]
X_test,y_test = X[train_ind:],y[train_ind:]

In [126]:
fm.fit(X_train, y_train)

FMRegression(l2_reg_V=0.5, n_iter=1000, rank=5)

In [127]:
y_pred = fm.predict(X_test)

In [128]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [129]:
mse_valid = MSE(y_pred, y_test)

In [130]:
mse_valid

1.2628854652560522

In [131]:
y_pred = fm.predict(X_train)

In [132]:
mse_train = MSE(y_pred, y_train)


In [133]:
mse_train

0.05196578173884204

In [134]:
y_train.shape

(359189,)

In [14]:
userAverages = {}
itemAverages = {}

usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set)
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair

for ind, d in df_interactions_train.iterrows():
    # print(d['u'], d['items'], d['ratings'])
    u,i,r = d['user_id'], d['recipe_id'], d['rating']
    usersPerItem[i].add(u)
    itemsPerUser[u].add(i)
    ratingDict[(u,i)] = r

for ind, d in df_interactions_valid.iterrows():
    # print(d['u'], d['items'], d['ratings'])
    u,i,r = d['user_id'], d['recipe_id'], d['rating']
    usersPerItem[i].add(u)
    itemsPerUser[u].add(i)
    ratingDict[(u,i)] = r

for ind, d in df_interactions_test.iterrows():
    # print(d['u'], d['items'], d['ratings'])
    u,i,r = d['user_id'], d['recipe_id'], d['rating']
    usersPerItem[i].add(u)
    itemsPerUser[u].add(i)
    ratingDict[(u,i)] = r

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [15]:
import scipy

userIDs,itemIDs = {},{}

for ind, d in df_interactions_train.iterrows():
    u,i = d['user_id'], d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
for ind, d in df_interactions_valid.iterrows():
    u,i = d['user_id'], d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
for ind, d in df_interactions_test.iterrows():
    u,i = d['user_id'], d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
nUsers = len(userIDs)
nItems = len(itemIDs)

In [16]:
X = scipy.sparse.lil_matrix((len(df_interactions_train)+len(df_interactions_valid)+len(df_interactions_test), nUsers+nItems+1))

In [34]:
for i in range (len(df_interactions_train)):
    user = userIDs[df_interactions_train.loc[i]['user_id']]
    item = itemIDs[df_interactions_train.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i,-1] = userAverages[df_interactions_train.loc[i]['user_id']]
for i in range (len(df_interactions_valid)):
    user = userIDs[df_interactions_valid.loc[i]['user_id']]
    item = itemIDs[df_interactions_valid.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i,-1] = userAverages[df_interactions_valid.loc[i]['user_id']]
for i in range (len(df_interactions_test)):
    user = userIDs[df_interactions_test.loc[i]['user_id']]
    item = itemIDs[df_interactions_test.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i,-1] = userAverages[df_interactions_test.loc[i]['user_id']]

In [35]:
import numpy as np
y = np.array([d['rating'] for _,d in df_interactions_train.iterrows()])

In [36]:
from fastFM import als
from scipy.spatial import distance
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)

In [37]:
train_ind = int(X.shape[0]*0.5)
train_ind

359189

In [38]:
X_train,y_train = X[:train_ind],y[:train_ind]
X_test,y_test = X[train_ind:],y[train_ind:]

In [39]:
fm.fit(X_train, y_train)

FMRegression(l2_reg_V=0.5, n_iter=1000, rank=5)

In [40]:
y_pred = fm.predict(X_test)

In [41]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [42]:
mse_valid = MSE(y_pred, y_test)

In [43]:
mse_valid

1.248179758401997

In [44]:
y_pred = fm.predict(X_train)

In [45]:
mse_train = MSE(y_pred, y_train)

In [46]:
mse_train

0.0519023063340347

In [47]:
for i in range (len(df_interactions_train)):
    user = userIDs[df_interactions_train.loc[i]['user_id']]
    item = itemIDs[df_interactions_train.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i,-1] = itemAverages[df_interactions_train.loc[i]['recipe_id']]
for i in range (len(df_interactions_valid)):
    user = userIDs[df_interactions_valid.loc[i]['user_id']]
    item = itemIDs[df_interactions_valid.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i,-1] = itemAverages[df_interactions_valid.loc[i]['recipe_id']]
for i in range (len(df_interactions_test)):
    user = userIDs[df_interactions_test.loc[i]['user_id']]
    item = itemIDs[df_interactions_test.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i,-1] = itemAverages[df_interactions_test.loc[i]['recipe_id']]

In [48]:
import numpy as np
y = np.array([d['rating'] for _,d in df_interactions_train.iterrows()])

In [49]:
from fastFM import als
from scipy.spatial import distance
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)

In [51]:
train_ind = int(X.shape[0]*0.7)
train_ind

502865

In [52]:
X_train,y_train = X[:train_ind],y[:train_ind]
X_test,y_test = X[train_ind:],y[train_ind:]

In [53]:
fm.fit(X_train, y_train)

FMRegression(l2_reg_V=0.5, n_iter=1000, rank=5)

In [54]:
y_pred = fm.predict(X_test)

In [55]:
mse_valid = MSE(y_pred, y_test)


In [56]:
mse_valid


1.4054928199852132

In [57]:
y_pred = fm.predict(X_train)

In [58]:
mse_train = MSE(y_pred, y_train)


In [59]:
mse_train


0.07203658851610605

In [76]:
df_raw_recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [93]:
upper = df_raw_recipes['minutes'].quantile(0.97)
med = int(np.median(df_raw_recipes['minutes']))

In [94]:
recipe_time = {}
for i, d in df_raw_recipes.iterrows():
    if d['minutes'] > upper:
        recipe_time[d['id']] = med
    else:
        recipe_time[d['id']] = d['minutes']

In [104]:
nMins = max(recipe_time.values()) - min(recipe_time.values())+1

In [105]:
X = scipy.sparse.lil_matrix((len(df_interactions_train)+len(df_interactions_valid)+len(df_interactions_test), nUsers+nItems+nMins))

In [106]:
for i in range (len(df_interactions_train)):
    user = userIDs[df_interactions_train.loc[i]['user_id']]
    item = itemIDs[df_interactions_train.loc[i]['recipe_id']]
    mins = recipe_time[df_interactions_train.loc[i]['recipe_id']]
    # print(mins)
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i, nUsers+nItems+mins] = 1
    
for i in range (len(df_interactions_valid)):
    user = userIDs[df_interactions_valid.loc[i]['user_id']]
    item = itemIDs[df_interactions_valid.loc[i]['recipe_id']]
    mins = recipe_time[df_interactions_valid.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i, nUsers+nItems+mins] = 1
    
for i in range (len(df_interactions_test)):
    user = userIDs[df_interactions_test.loc[i]['user_id']]
    item = itemIDs[df_interactions_test.loc[i]['recipe_id']]
    mins = recipe_time[df_interactions_test.loc[i]['recipe_id']]
    X[i,user] = 1
    X[i, nUsers+item] = 1
    X[i, nUsers+nItems+mins] = 1
    

In [107]:
y = np.array([d['rating'] for _,d in df_interactions_train.iterrows()])

In [119]:
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=10, l2_reg_w=0.1, l2_reg_V=0.5)

In [120]:
train_ind = int(X.shape[0]*0.7)
train_ind

502865

In [121]:
X_train,y_train = X[:train_ind],y[:train_ind]
X_test,y_test = X[train_ind:],y[train_ind:]

In [122]:
fm.fit(X_train, y_train)


FMRegression(l2_reg_V=0.5, n_iter=1000, rank=10)

In [123]:
y_pred = fm.predict(X_test)


In [124]:
mse_valid = MSE(y_pred, y_test)

In [125]:
mse_valid

1.486069633713466

In [126]:
y_pred = fm.predict(X_train)

In [127]:
mse_train = MSE(y_pred, y_train)

In [128]:
mse_train

0.022146394128876347